In [29]:
import math

outputs = [1,1,1,1,1,1,1,0,0,0]
inputs = [[0,1,1], [1,1,1], [1, 0, 0], [1,0,1], [1,0,1], [0,1,1], [0,1,1], 
              [0,1,0], [1,0,0], [0,0,1]]

#attributes: evil, transforms, sparkly, TRUE, FALSE
stumps = [{'Y': [1, 6, 7, 9], 'N': [2, 3, 4, 5, 8, 10]}, 
                 {'Y': [3,4,5,8], 'N': [1,2,6,7,9,10]},
                 {'Y': [3, 10], 'N': [1,2,4,5,6,7,8,9]}, 
                 {'Y': [8,9,10], 'N': [1,2,3,4,5,6,7]}]

def gen_stumps(inputs):
    stumps = [] 
    
    #first we identify which values are possible
    for attr in range(len(inputs[0])):
        stumps.append({})
        for line in inputs:
            value = line[attr]
            stumps[attr][value] = []
    
    #now we generate the stumps
    for (attr, stump) in enumerate(stumps):
        for (category, wrong_elements) in stump.items():
            for (i, line) in enumerate(inputs):
                if category == line[attr] and outputs[i] == 0 or category != line[attr] and outputs[i] == 1:
                    wrong_elements.append(i)
                    
    #extra stump to represent that everything is 1 or everything is 0
    extra_stump = {0: [], 1: []}
    for (i, output) in enumerate(outputs):
        extra_stump[1 if output == 0 else 0].append(i)
    
    stumps.append(extra_stump)
    
    return stumps

def boosting(inputs, outputs, num_iters):
    model = []
    stumps = gen_stumps(inputs)
    
    for i in range(num_iters):
        model.append(boost_round(stumps, weights))
    
    return model

def predict(model, input):
    for 

def boost_round(stumps, weights):
    best_error = 1000 #this is correct, since the max total error is 1
    best_stump = None
    wrongs = []
    
    for attribute, categories in enumerate(stumps):
        for (category_value, wrong_elements) in categories.items():
            error = 0
            for id in wrong_elements:
                error += weights[id]
            if error < best_error:
                best_error = error
                best_stump = (attribute, category_value)
                wrongs = wrong_elements
    
    alpha = 1/2 * math.log((1 - best_error) / best_error)
    
    sum = 0
    for i, wold in enumerate(weights):
        #if i == 0: continue
        factor = -1 if i in wrongs else 1
        wnew = wold * math.exp(-alpha * factor)
        weights[i] = wnew
        sum += wnew
    
    for i, weight in enumerate(weights):
        weights[i] = weight / sum
        
    return (alpha, best_stump)
          
weights = [0, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]
res = boost_round(stumps, weights) 
print(res)
print(weights)
res = boost_round(stumps, weights) 
print(res)
print(weights)
res = boost_round(stumps, weights) 
print(res)
print(weights)

(0.6931471805599453, (2, 'Y'))
[0.0, 0.0625, 0.0625, 0.25, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.0625, 0.25]
(0.5493061443340549, (0, 'Y'))
[0.0, 0.125, 0.041666666666666664, 0.16666666666666666, 0.041666666666666664, 0.041666666666666664, 0.125, 0.125, 0.041666666666666664, 0.125, 0.16666666666666666]
(0.44365159750045147, (1, 'Y'))
[0.0, 0.08823529411764705, 0.02941176470588235, 0.2857142857142857, 0.07142857142857142, 0.07142857142857142, 0.08823529411764705, 0.08823529411764705, 0.07142857142857142, 0.08823529411764705, 0.1176470588235294]


In [30]:
boosting(inputs, outputs, 3)

1.0074515102711323
(2, 1)
0.9462820841750103
(0, 1)
0.6412549449587649
(1, 1)
